In [1]:
import os
import pdf2image
from pdf2image import convert_from_path
import cv2
import matplotlib.pyplot as plt
import PyPDF2
import PIL
from PIL import Image, ImageTk
import numpy as np
import pytesseract
from pytesseract import Output
pytesseract.pytesseract.tesseract_cmd="/home/aman/anaconda3/envs/tallyInvoiceParser.env/bin/tesseract"
os.environ['TESSDATA_PREFIX'] = "/home/aman/anaconda3/envs/tallyInvoiceParser.env/share/tessdata"
os.environ['MLIR_CRASH_REPRODUCER_DIRECTORY']='tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269'
import import_ipynb
from imagePreProcessor import *
from CreateDatasetForDateModel import *
from TesseractModify import *
from datetime import datetime
import math
from keras.models import load_model
import time
import sys
import psutil
from tqdm import tqdm
import datefinder
import dateparser
from IPython.display import clear_output

importing Jupyter notebook from imagePreProcessor.ipynb
importing Jupyter notebook from CreateDatasetForDateModel.ipynb


2023-06-15 15:57:51.916587: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


importing Jupyter notebook from TesseractModify.ipynb


In [2]:
def preProcessImage(image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    blur = cv2.GaussianBlur(gray, (3,3),1)
    threshold_img = cv2.adaptiveThreshold(blur,255,1,1,11,2)
    return threshold_img

In [3]:
def processForInput(df):
    colsToDrop = ["left", "top", "width", "height", "text", "index", "PageHeight", "PageWidth", "conf", "block_num", "line_num", "isPast", "isFuture", "nextDist"]
    d = df.drop(columns=colsToDrop)
    d["x"]=d["x"]/1000
    d["y"]=d["y"]/1000
    return d.to_numpy()

In [4]:
def predictPdf(filePath):
    f = open(filePath, 'rb')
    readpdf = PyPDF2.PdfReader(f)
    totalpages = len(readpdf.pages)
    if totalpages==1:
        image = convert_from_path(filePath)
    else:
        return
    image = np.array(image[0])
    image = preProcessImage(image)
    data = pytesseract.image_to_data(image, output_type=Output.DATAFRAME)
    data.dropna(inplace=True)
    data.reset_index(inplace=True)
    data.drop(columns=["index"], inplace=True)
    processData(data)
    df = pd.DataFrame(data)
    preProcessDataFrame(df, image)
    dateDF, dates = extractDateDataFrame(df)
    addNeighbours(df)
    indexes = findDateDF(dates, dateDF)
    resultDF = df.copy()
    dropIndexes(indexes, resultDF)
    resultDF.reset_index(inplace=True)
    texts = []
    inputs = []
    for index in resultDF.index:
        texts.append(resultDF["text"][index])
        inputs.append(processForInput(resultDF))
    predictions = {}
    model = load_model('/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Tesseract-Modelv5/Models/tesseractv5.h5')
    for i in range(len(inputs)):
        text = texts[i]
        input = inputs[i]
        pred = model.predict(input)
        predictions[text]=max(pred)
    if len(predictions)==0:
        return None

    res = max(zip(predictions.values(), predictions.keys()))[1]
    return res
    

In [5]:
pwd

'/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Tesseract-Modelv5'

In [6]:
os.chdir('/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Tesseract-Modelv5/Dataset/SampleDocuments')

In [7]:
dir = os.listdir('/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Tesseract-Modelv5/Dataset/SampleDocuments')

In [8]:
pdfPath = "7958400.pdf"

In [9]:
result = predictPdf(pdfPath)
dates = list(datefinder.find_dates(result, strict=True))
if len(dates)==0:
    print("Sorry, invoice date not found")
else:
    date = dates[0]
    day = date.day
    month = date.month
    year = date.year
    invDate = str(day) + "/" + str(month) + "/" + str(year)
    print("Invoice Date : " + invDate)
    

<string>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<string>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


[datetime.datetime(2022, 9, 28, 0, 0)]
[datetime.datetime(2022, 9, 28, 0, 0)]


<string>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<string>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


1/1 [==============================] - 0s 18ms/step
Invoice Date : 28/9/2022


In [10]:
def predictImage(imagePath):
    image = cv2.imread(imagePath)
    image = preProcessImage(image)
    data = pytesseract.image_to_data(image, output_type=Output.DATAFRAME)
    data.dropna(inplace=True)
    data.reset_index(inplace=True)
    data.drop(columns=["index"], inplace=True)
    processData(data)
    df = pd.DataFrame(data)
    preProcessDataFrame(df, image)
    dateDF, dates = extractDateDataFrame(df)
    addNeighbours(df)
    indexes = findDateDF(dates, dateDF)
    resultDF = df.copy()
    dropIndexes(indexes, resultDF)
    resultDF.reset_index(inplace=True)
    texts = []
    inputs = []
    for index in resultDF.index:
        texts.append(resultDF["text"][index])
        inputs.append(processForInput(resultDF))
    predictions = {}
    model = load_model('/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Tesseract-Modelv5/Models/tesseractv5.h5')
    for i in range(len(inputs)):
        text = texts[i]
        input = inputs[i]
        pred = model.predict(input)
        predictions[text]=max(pred)
    if len(predictions)==0:
        return None
    res = max(zip(predictions.values(), predictions.keys()))[1]
    return res

In [11]:
pwd

'/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Tesseract-Modelv5/Dataset/SampleDocuments'

In [12]:
os.chdir('/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Tesseract-Modelv5/Dataset/SampleImages/')

In [13]:
start = time.time()
result = predictImage("3.jpeg")
dates = list(datefinder.find_dates(result, strict=True))
if len(dates)==0:
    print("Sorry, invoice date not found")
else:
    date = dates[0]
    day = date.day
    month = date.month
    year = date.year
    invDate = str(day) + "/" + str(month) + "/" + str(year)
    clear_output(wait=True)
    print("Invoice Date : " + invDate)
end = time.time()
print("\n\nTotal Time Taken : " + str(end-start) + " seconds")

Invoice Date : 9/10/2022


Total Time Taken : 1.6644105911254883 seconds


In [14]:
def benchmark():
    
    total_steps = 1000
    progress_bar = tqdm(total=total_steps, ncols=80)
    
    n = input("Enter number of iterations : ")
    
    
    start = time.time()
    for i in range(int(n)):
        predictImage("/home/aman/Desktop/4.jpeg")
    end = time.time()
    
    inferenceTime = end-start
    
    cpuUsage = []
    memoryUsage = []
    
    start_time = time.time()
    step=1
    for run in range(10):
        cpuList = []
        memoryList = []
        process = psutil.Process()

        for i in range(10):
            cpuPercent = process.cpu_percent(interval=0.01)
            memory = process.memory_info().rss
            cpuList.append(cpuPercent)
            memoryList.append(memory)

            for j in range(int(n)):
                predictImage("/home/aman/Desktop/4.jpeg")
                
                progress_bar.update(1)
                elapsed_time = time.time() - start_time
                remaining_time = elapsed_time / (step + 1) * (total_steps - step - 1)
                progress_bar.set_postfix({'ETA': progress_bar.format_interval(remaining_time)})
                step+=1
                
        avgCPU = sum(cpuList) / len(cpuList)
        cpuUsage.append(avgCPU)
        avgMemory = sum(memoryList) / len(memoryList)
        memoryUsage.append(avgMemory)
    
    maxCPU = max(cpuUsage)
    minCPU = min(cpuUsage)
    
    maxMemory = max(memoryUsage)
    minMemory = min(memoryUsage)
    
    progress_bar.close()
    
    
    return inferenceTime, maxCPU, minCPU, maxMemory, minMemory
    
    

In [15]:
#inferenceTime, maxCPU, minCPU, maxMemory, minMemory = benchmark()

In [16]:
print("TIME TAKEN : ", inferenceTime)
print("MAX CPU : ", maxCPU, "%")
print("MAX MEMORY : ", maxMemory//(1024*1024), "MB")
print("MIN MEMORY : ", minMemory//(1024*1024), "MB")

NameError: name 'inferenceTime' is not defined

In [17]:
pwd

'/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Tesseract-Modelv5/Dataset/SampleImages'

In [18]:
imagesDir = os.listdir('/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Tesseract-Modelv5/Dataset/SampleImages')

In [19]:
times = []
progress = []
for image in imagesDir:
    print(image)
    L = []
    start = time.time()
    result = predictImage(image)
    if result == None:
        dates = []
    else:
        dates = list(datefinder.find_dates(result, strict=True))
    if len(dates)==0:
        L.append("Sorry, invoice date not found")
    else:
        date = dates[0]
        day = date.day
        month = date.month
        year = date.year
        invDate = str(day) + "/" + str(month) + "/" + str(year)
        print("Invoice Date : " + invDate)
        L.append(invDate)
        
    end = time.time()
    times.append(end-start)
    L.append(end-start)
    progress.append(L)
    
clear_output(wait=True)

for L in progress:
    invDate = L[0]
    T = L[1]
    print("Invoice Date : " + str(invDate))
    print("Time Taken : " + str(T))
    print("\n")
    
print("\n\n")
print("Max Time Taken : " + str(max(times)))
print("Min Time Taken : " + str(min(times)))
print("Avg Time Taken : " + str(sum(times)/len(times)))

Invoice Date : 4/2/2023
Time Taken : 4.982791423797607


Invoice Date : 10/9/2022
Time Taken : 2.288965940475464


Invoice Date : 5/12/2022
Time Taken : 2.0505967140197754


Invoice Date : 14/4/2022
Time Taken : 2.454124689102173


Invoice Date : 17/4/2022
Time Taken : 3.4239933490753174


Invoice Date : Sorry, invoice date not found
Time Taken : 2.1805572509765625


Invoice Date : 9/12/2022
Time Taken : 2.5120129585266113


Invoice Date : 12/5/2022
Time Taken : 3.1272284984588623


Invoice Date : 12/1/2022
Time Taken : 2.404514789581299


Invoice Date : 12/5/2022
Time Taken : 4.287399530410767


Invoice Date : 21/8/2022
Time Taken : 2.796804189682007


Invoice Date : 9/10/2022
Time Taken : 1.6487743854522705


Invoice Date : 7/3/2023
Time Taken : 5.808650016784668


Invoice Date : 16/12/2022
Time Taken : 4.502142429351807


Invoice Date : Sorry, invoice date not found
Time Taken : 6.353074550628662


Invoice Date : 30/1/2023
Time Taken : 6.402421712875366


Invoice Date : 21/12/2022
T

In [54]:
imagePath ='/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Tesseract-Modelv5/Dataset/Testing/5.jpeg'

In [55]:
result = predictImage(imagePath)

<string>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<string>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<string>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<string>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


[datetime.datetime(2023, 4, 25, 16, 58)]
1/1 [==============================] - 0s 37ms/step


In [56]:
print(result)

April 25, 2023 at 04:58 PM 
